In [ ]:
import pandas as pd
import numpy as np

In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 79.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 116.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.8/199.8 KB 25.2 MB/s eta 0:00:00


In [ ]:
df.columns

Index(['Unnamed: 0', 'id', 'comment_text', 'toxic', 'severe_toxic', 'obscene',
       'threat', 'insult', 'identity_hate'],
      dtype='object')

In [ ]:
df = pd.read_csv("/content/toxic_comment (1).csv")



In [ ]:
df = df[["comment_text","toxic"]]

In [ ]:
df.sample(frac=1)
df.reset_index()

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score,accuracy_score,precision_score,recall_score

from transformers import TrainingArguments , Trainer   #train use trainerarguments
import torch
from transformers import BertTokenizer,BertForSequenceClassification

In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForSequenceClassification.from_pretrained('bert-base-uncased',num_labels = 2)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [ ]:
sample = ["Currently i am studing BERT model which is a transformer based mdoel"]
token = tokenizer(sample,padding=True,truncation=True , max_length = 512)

In [ ]:
token

{'input_ids': [[101, 2747, 1045, 2572, 16054, 2075, 14324, 2944, 2029, 2003, 1037, 10938, 2121, 2241, 9108, 8913, 2140, 102]], 'token_type_ids': [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]}

In [ ]:
X = list(df["comment_text"])
y = list(df['toxic'])

X_train , X_test , y_train , y_test = train_test_split(X,y,test_size=0.2,stratify=y)

X_train_token = tokenizer(X_train,padding = True,max_length=512,truncation=True)
X_test_token = tokenizer(X_train,padding = True,max_length=512,truncation=True)

In [ ]:
model

## **Create Dataset**

Turn our labels and encodings into a Dataset object. In PyTorch, this is done by subclassing a torch.utils.data.Dataset object and implementing `__len__` and `__getitem__`. In TensorFlow, we pass our input encodings and labels to the from_tensor_slices constructor method.

We put the data in this format so that the data can be easily batched such that each key in the batch encoding corresponds to a named parameter of the forward() method of the model we will train.

In [ ]:
# Create torch dataset
class Dataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels=None):
        self.encodings = encodings
        self.labels = labels       #it takes out the output can corrosponding labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        if self.labels:
            item["labels"] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.encodings["input_ids"])  #length of input_id

        #masked id show how much word it find relevant and how much word it masked  

In [ ]:
train_dataset = Dataset(X_train_token, y_train)
val_dataset = Dataset(X_test_token, y_test)

In [ ]:
X_train[0]

'"\nMaybe you should look up what harassment is.  Please change the reasoning for my block to something respectable that deals with the situation at hand.  I did not harrass you in any way, shape, or form.  I asked you to fuck off in the context of my user talk page.  And I repeat again my statement, fuck off my talk page when this issue is over.  It is my right to ask you to fuck off my talk page.  It is not my right to block you from this page, but again it is my right to ask you to fuck off this user talk page.  Bits andPieces \nAnd why will you have to do that?  I am disrupting the project by asking you to fuck off my talk page?  Nope.   Bits andPieces \nLengthened the block to 1 week, and you can\'t edit this page. I hope you are more reasonable next week, see you then!  // have a cup // ark //  "'

In [ ]:
X_train_token['input_ids'][0]

In [ ]:
train_dataset[0]

{'input_ids': tensor([  101,  1000,  2672,  2017,  2323,  2298,  2039,  2054, 16011,  2003,
          1012,  3531,  2689,  1996, 13384,  2005,  2026,  3796,  2000,  2242,
         19416,  2008,  9144,  2007,  1996,  3663,  2012,  2192,  1012,  1045,
          2106,  2025,  5292, 11335,  4757,  2017,  1999,  2151,  2126,  1010,
          4338,  1010,  2030,  2433,  1012,  1045,  2356,  2017,  2000,  6616,
          2125,  1999,  1996,  6123,  1997,  2026,  5310,  2831,  3931,  1012,
          1998,  1045,  9377,  2153,  2026,  4861,  1010,  6616,  2125,  2026,
          2831,  3931,  2043,  2023,  3277,  2003,  2058,  1012,  2009,  2003,
          2026,  2157,  2000,  3198,  2017,  2000,  6616,  2125,  2026,  2831,
          3931,  1012,  2009,  2003,  2025,  2026,  2157,  2000,  3796,  2017,
          2013,  2023,  3931,  1010,  2021,  2153,  2009,  2003,  2026,  2157,
          2000,  3198,  2017,  2000,  6616,  2125,  2023,  5310,  2831,  3931,
          1012,  9017,  1998, 11198,  2

from sentences tokenizer tokenize to numbers and in that it contain four things  input id , token_type_id , attention_mask
and labels so these for each row this data is created which would form like a dataset

In [ ]:
def compute_metrics(p):
    print(type(p))
    pred, labels = p   #transformer evaluation output has two things one which is labels and second which is prediciton
    pred = np.argmax(pred, axis=1) #so take out only labels
    #then basic sklearn matrices
    accuracy = accuracy_score(y_true=labels, y_pred=pred)
    recall = recall_score(y_true=labels, y_pred=pred)
    precision = precision_score(y_true=labels, y_pred=pred)
    f1 = f1_score(y_true=labels, y_pred=pred)

    return {"accuracy": accuracy, "precision": precision, "recall": recall, "f1": f1}

In [ ]:
# Define Trainer
args = TrainingArguments(
    output_dir="output",
    num_train_epochs=20,
    per_device_train_batch_size=8

)
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics
)

In [ ]:
trainer.train()

Step,Training Loss
500,0.112200
1000,0.002300
1500,0.000000
2000,0.000000


TrainOutput(global_step=2000, training_loss=0.028627024253830315, metrics={'train_runtime': 1435.1465, 'train_samples_per_second': 11.149, 'train_steps_per_second': 1.394, 'total_flos': 4209776885760000.0, 'train_loss': 0.028627024253830315, 'epoch': 20.0})

In [ ]:
trainer.evaluate()

In [ ]:
np.set_printoptions(suppress=True)

In [ ]:
text = "That was good point"
# text = "go to hell"
inputs = tokenizer(text,padding = True, truncation = True, return_tensors='pt').to('cuda')  #tokenize the input
outputs = model(**inputs)  #predict the output
print(outputs)
predictions = torch.nn.functional.softmax(outputs.logits, dim=-1)   #output from the models arae logits and convert those logits to softmax probabilities
print(predictions)
predictions = predictions.cpu().detach().numpy() #convert those prediction to numpy array
predictions

SequenceClassifierOutput(loss=None, logits=tensor([[ 5.1543, -4.1925]], device='cuda:0', grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)
tensor([[9.9991e-01, 8.7240e-05]], device='cuda:0', grad_fn=<SoftmaxBackward0>)


array([[0.99991274, 0.00008724]], dtype=float32)

In [ ]:
trainer.save_model('CustomModel')

In [ ]:
model_2 = BertForSequenceClassification.from_pretrained("CustomModel")
model_2.to('cuda')

Lets summerize what we have done above 
 * Take the data and divide it into train and test .
 * Load the libraries and transformer model with his tokenizer . 
 * Tokenize the training and testing data . 
 * Create a dataset object using pytorch because this dataset is supported by your trainer
 * Define your trainer and trainer argument and evaluation metrics
 * Train and Evaluate your model with Trainer 
 * Save and laod your pretrained model or you can also host in hugging face

**Deploy fined tuned bert model on hugging face**

* Push model to hugging face 
* Deploy it on any cloud platform

Hugging face is like githug for ml model . Suppose you fined tuned your bert model now you want to dockerize it so if you use that model locally and create its image then the image of that file would of very large and while transfering the docker image to AWS site would be difficult

Suppose we have to re trained or update our model then if any updatation is done then we have to again dockerize the whole model from scratch but if we push our model to hugging face then we can load that model just like any pretrained models without making any image of that model

Next time when you update your model and push to hugging face it will be reflected on your model without any issue or updatation in main app. 

In [ ]:
!pip install transformers 

In [ ]:
from huggingface_hub import notebook_login

notebook_login()
     

Token is valid.
Your token has been saved in your configured git credential helpers (store).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [ ]:
model.push_to_hub("Amite5h/BERTfinedTunedontoxic")

pytorch_model.bin:   0%|          | 0.00/438M [00:00<?, ?B/s]

Upload 1 LFS files:   0%|          | 0/1 [00:00<?, ?it/s]

CommitInfo(commit_url='https://huggingface.co/Amite5h/BERTfinedTunedontoxic/commit/8e55f40328d47dfb2f72665bb32e254c147ff994', commit_message='Upload BertForSequenceClassification', commit_description='', oid='8e55f40328d47dfb2f72665bb32e254c147ff994', pr_url=None, pr_revision=None, pr_num=None)

In [ ]:
model = BertForSequenceClassification.from_pretrained("Your git hub id",use_auth_token="")   #for private model

In [ ]:
#after uploading to hugging face in file and verison you will see config and your model , config is necessary

#since it is available on githug you can load it as any other model 